In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mmap
import random
import pickle
import argparse

#parser = argparse.ArgumentParser(description='This is a demo program')
#parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')
#args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

batch_size = 32
block_size = 128
max_iters = 10000
learning_rate = 1e-4 #3e-3, 3e-4, 1e-3, 1e-4
eval_iters = 100
n_embd = 384
n_head = 16
n_layer = 16 #number of blocks
dropout = 0.2

cuda


In [2]:
chars = ""
with open('vocab.txt','r', encoding='utf-8') as f:
    text  = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)
print(vocab_size)

32172


In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


In [4]:
def get_random_chunk(split):
    filename = "train_split.txt" if split == 'train' else "val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[  75,   86,   75,  ...,   86,   75,   69],
        [   0,    0,    0,  ...,   85, 6187,   14],
        [  85,   74,   71,  ...,   74,   67,   86],
        ...,
        [  73,   74,   86,  ...,   81,    2,   72],
        [  84,   85,   86,  ...,   69,   71,    2],
        [  67,   68,   81,  ...,    2,   35,   79]], device='cuda:0')
targets:
tensor([[  86,   75,   88,  ...,   75,   69,   87],
        [   0,    0,   38,  ..., 6187,   14,    2],
        [  74,   71,   70,  ...,   67,   86,   16],
        ...,
        [  74,   86,   71,  ...,    2,   72,   81],
        [  85,   86,   67,  ...,   71,    2,   86],
        [  68,   81,   87,  ...,   35,   79,   71]], device='cuda:0')


In [5]:
class Head(nn.Module):
    #one head of self attention
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self,x ):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #Compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 #(B,T,hs) @ (B,hs,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1) #(B,T,T)
        wei = self.dropout(wei)
        #perform weighted aggregation of the values
        v = self.value(x) #(B,T,hs)
        out = wei @ v #(B,T,T) @ (B,T,hs) -> (B,T,hs)
        return out    

class MultiHeadAttention(nn.Module):
    # Multiple heads of self attention in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x ):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #(B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    #Simple layer followed by non-linearity
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd),
                                 nn.ReLU(),
                                 nn.Linear(4*n_embd, n_embd),
                                 nn.Dropout(dropout),
                                )

    def forward(self, x):
        return self.net(x)
    
        
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) #Self Attention
        self.ffwd = FeedForward(n_embd) #Feed Forward
        self.ln1 = nn.LayerNorm(n_embd) #Layer Norm 1
        self.ln2 = nn.LayerNorm(n_embd) #Layer Norm 2

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_emdedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
               
        B, T = index.shape

        # index and targets are both (B,T) tensor of integers        
        tok_emb = self.token_embedding_table(index) #(B,T,C)
        pos_emb = self.position_emdedding_table(torch.arange(T, device=device))#(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.blocks(x) #(B,T,C)
        x = self.ln_f(x) #(B,T,C)
        logits = self.lm_head(x) #(B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = GPTLanguageModel(vocab_size)

print('loading model parameters...')
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)
print('model loaded successfully')
m = model.to(device)

loading model parameters...
model loaded successfully


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.3f}, val loss : {losses['train']:.3f}')
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item()) 

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step: 0, train loss: 1.255, val loss : 1.255
step: 100, train loss: 1.234, val loss : 1.234
step: 200, train loss: 1.246, val loss : 1.246
step: 300, train loss: 1.281, val loss : 1.281
step: 400, train loss: 1.205, val loss : 1.205
step: 500, train loss: 1.314, val loss : 1.314
step: 600, train loss: 1.230, val loss : 1.230
step: 700, train loss: 1.262, val loss : 1.262
step: 800, train loss: 1.241, val loss : 1.241
step: 900, train loss: 1.236, val loss : 1.236
step: 1000, train loss: 1.227, val loss : 1.227
step: 1100, train loss: 1.268, val loss : 1.268
step: 1200, train loss: 1.226, val loss : 1.226
step: 1300, train loss: 1.263, val loss : 1.263
step: 1400, train loss: 1.215, val loss : 1.215
step: 1500, train loss: 1.260, val loss : 1.260
step: 1600, train loss: 1.215, val loss : 1.215
step: 1700, train loss: 1.241, val loss : 1.241
step: 1800, train loss: 1.257, val loss : 1.257
step: 1900, train loss: 1.263, val loss : 1.263
step: 2000, train loss: 1.287, val loss : 1.287
step

In [8]:
prompt = 'define anarchists '
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars=decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

define anarchists absurd. Guto a qualification of game, the active fundamental problem was now intervened closing and 
